In [1]:
# import necessary packages
import super_gradients

[2023-06-29 12:41:29] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it


The console stream is logged into /home/computervision_zl/sg_logs/console.log


/usr/local/lib/python3.8/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# initialize trainer
from super_gradients import Trainer
from super_gradients.training import MultiGPUMode
from super_gradients.training.metrics import DetectionMetrics_050

In [3]:
# importing models from supergradients
from super_gradients.training import models

In [4]:
# instantiate the dataset
from super_gradients.training.datasets.detection_datasets.coco_format_detection import COCOFormatDetectionDataset
from super_gradients.training.transforms.transforms import DetectionMosaic, DetectionRandomAffine, DetectionHSV, \
    DetectionHorizontalFlip, DetectionPaddedRescale, DetectionStandardize, DetectionTargetsFormatTransform
from super_gradients.training.utils.detection_utils import DetectionCollateFN
from super_gradients.training import dataloaders
from super_gradients.training.datasets.datasets_utils import worker_init_reset_seed


testset = COCOFormatDetectionDataset(data_dir="dataset",
                                     images_dir="images/test",
                                     json_annotation_file="annotations/test_coco.json",
                                     input_dim=(640, 640),
                                     ignore_empty_annotations=False,
                                     transforms=[
                                         DetectionPaddedRescale(input_dim=(640, 640), max_targets=300),
                                         DetectionStandardize(max_value=255),
                                         DetectionTargetsFormatTransform(max_targets=300, input_dim=(640, 640),
                                                                         output_format="LABEL_CXCYWH")
                                     ])

In [5]:
# configure dataloader

test_loader_8 = dataloaders.get(dataset=testset, dataloader_params={
    "shuffle": False,
    "batch_size": 8,
    "num_workers": 8,
    "drop_last": False,
    "pin_memory": True,
    "collate_fn": DetectionCollateFN(),
    "worker_init_fn": worker_init_reset_seed
})

test_loader_16 = dataloaders.get(dataset=testset, dataloader_params={
    "shuffle": False,
    "batch_size": 16,
    "num_workers": 8,
    "drop_last": False,
    "pin_memory": True,
    "collate_fn": DetectionCollateFN(),
    "worker_init_fn": worker_init_reset_seed
})

In [6]:
# training setup
from super_gradients.training import training_hyperparams
from super_gradients.training import dataloaders
# dataloader setup
from super_gradients.training.models.detection_models.pp_yolo_e import PPYoloEPostPredictionCallback
from pprint import pprint

from super_gradients.common.environment.cfg_utils import load_recipe


# Test

___________________

In [7]:
from time import sleep

# Hyperparameters
params = [
    {'batch': 8, 'lr': 0.001, 'optimizer': 'Adam'},
    {'batch': 8, 'lr': 0.001, 'optimizer': 'SGD'},
    {'batch': 8, 'lr': 0.01, 'optimizer': 'Adam'},
    {'batch': 8, 'lr': 0.01, 'optimizer': 'SGD'},
    {'batch': 16, 'lr': 0.001, 'optimizer': 'Adam'},
    {'batch': 16, 'lr': 0.001, 'optimizer': 'SGD'},
    {'batch': 16, 'lr': 0.01, 'optimizer': 'Adam'},
    {'batch': 16, 'lr': 0.01, 'optimizer': 'SGD'}
]

In [8]:
# Experiment 1    {'batch': 8, 'lr': 0.001, 'optimizer': 'Adam'}
model = models.get("yolo_nas_m", checkpoint_path="/home/computervision_zl/Documents/yolonas/notebook_ckpts/YOLONAS_1/ckpt_best.pth", num_classes=13)

CHECKPOINT_DIR = 'notebook_ckpts_test/'
trainer = Trainer(experiment_name=f'YOLONAS_1', ckpt_root_dir=CHECKPOINT_DIR)

print(
    trainer.test(
        model=model,
        test_loader=test_loader_8,
        test_metrics_list=DetectionMetrics_050(
            score_thres=0.1, 
            top_k_predictions=300, 
            num_cls=13, 
            normalize_targets=True, 
            post_prediction_callback=PPYoloEPostPredictionCallback(
                score_threshold=0.01, 
                nms_top_k=1000, 
                max_predictions=300,
                nms_threshold=0.7)
            )
        )
)

del trainer
sleep(5)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


Test: 100%|██████████| 64/64 [00:06<00:00, 10.34it/s]


{'Precision@0.50': tensor(0.1153), 'Recall@0.50': tensor(0.9464), 'mAP@0.50': tensor(0.7784), 'F1@0.50': tensor(0.1887)}


In [9]:
# Experiment 2     {'batch': 8, 'lr': 0.001, 'optimizer': 'SGD'}
model = models.get("yolo_nas_m", checkpoint_path="/home/computervision_zl/Documents/yolonas/notebook_ckpts/YOLONAS_2/ckpt_best.pth", num_classes=13)

CHECKPOINT_DIR = 'notebook_ckpts_test/'
trainer = Trainer(experiment_name=f'YOLONAS_2', ckpt_root_dir=CHECKPOINT_DIR)

print(
    trainer.test(
        model=model,
        test_loader=test_loader_8,
        test_metrics_list=DetectionMetrics_050(
            score_thres=0.1, 
            top_k_predictions=300, 
            num_cls=13, 
            normalize_targets=True, 
            post_prediction_callback=PPYoloEPostPredictionCallback(
                score_threshold=0.01, 
                nms_top_k=1000, 
                max_predictions=300,
                nms_threshold=0.7)
            )
        )
)

del trainer
sleep(5)

Test: 100%|██████████| 64/64 [00:04<00:00, 14.81it/s]


{'Precision@0.50': tensor(0.3574), 'Recall@0.50': tensor(0.9574), 'mAP@0.50': tensor(0.9064), 'F1@0.50': tensor(0.4976)}


In [10]:
# Experiment 3    {'batch': 8, 'lr': 0.01, 'optimizer': 'Adam'}
model = models.get("yolo_nas_m", checkpoint_path="/home/computervision_zl/Documents/yolonas/notebook_ckpts/YOLONAS_3/ckpt_best.pth", num_classes=13)

CHECKPOINT_DIR = 'notebook_ckpts_test/'
trainer = Trainer(experiment_name=f'YOLONAS_3', ckpt_root_dir=CHECKPOINT_DIR)

print(
    trainer.test(
        model=model,
        test_loader=test_loader_8,
        test_metrics_list=DetectionMetrics_050(
            score_thres=0.1, 
            top_k_predictions=300, 
            num_cls=13, 
            normalize_targets=True, 
            post_prediction_callback=PPYoloEPostPredictionCallback(
                score_threshold=0.01, 
                nms_top_k=1000, 
                max_predictions=300,
                nms_threshold=0.7)
            )
        )
)

del trainer
sleep(5)

Test: 100%|██████████| 64/64 [00:04<00:00, 14.33it/s]


{'Precision@0.50': tensor(0.0602), 'Recall@0.50': tensor(0.7476), 'mAP@0.50': tensor(0.3813), 'F1@0.50': tensor(0.1006)}


In [11]:
# Experiment 4    {'batch': 8, 'lr': 0.01, 'optimizer': 'SGD'}
model = models.get("yolo_nas_m", checkpoint_path="/home/computervision_zl/Documents/yolonas/notebook_ckpts/YOLONAS_4/ckpt_best.pth", num_classes=13)

CHECKPOINT_DIR = 'notebook_ckpts_test/'
trainer = Trainer(experiment_name=f'YOLONAS_4', ckpt_root_dir=CHECKPOINT_DIR)

print(
    trainer.test(
        model=model,
        test_loader=test_loader_8,
        test_metrics_list=DetectionMetrics_050(
            score_thres=0.1, 
            top_k_predictions=300, 
            num_cls=13, 
            normalize_targets=True, 
            post_prediction_callback=PPYoloEPostPredictionCallback(
                score_threshold=0.01, 
                nms_top_k=1000, 
                max_predictions=300,
                nms_threshold=0.7)
            )
        )
)

del trainer
sleep(5)

Test: 100%|██████████| 64/64 [00:04<00:00, 14.62it/s]


{'Precision@0.50': tensor(0.1637), 'Recall@0.50': tensor(0.9487), 'mAP@0.50': tensor(0.8288), 'F1@0.50': tensor(0.2607)}


In [12]:
# Experiment 5    {'batch': 16, 'lr': 0.001, 'optimizer': 'Adam'}
model = models.get("yolo_nas_m", checkpoint_path="/home/computervision_zl/Documents/yolonas/notebook_ckpts/YOLONAS_5/ckpt_best.pth", num_classes=13)

CHECKPOINT_DIR = 'notebook_ckpts_test/'
trainer = Trainer(experiment_name=f'YOLONAS_5', ckpt_root_dir=CHECKPOINT_DIR)

print(
    trainer.test(
        model=model,
        test_loader=test_loader_16,
        test_metrics_list=DetectionMetrics_050(
            score_thres=0.1, 
            top_k_predictions=300, 
            num_cls=13, 
            normalize_targets=True, 
            post_prediction_callback=PPYoloEPostPredictionCallback(
                score_threshold=0.01, 
                nms_top_k=1000, 
                max_predictions=300,
                nms_threshold=0.7)
            )
        )
)

del trainer
sleep(5)

Test: 100%|██████████| 32/32 [00:04<00:00,  6.46it/s]


{'Precision@0.50': tensor(0.1378), 'Recall@0.50': tensor(0.9537), 'mAP@0.50': tensor(0.7969), 'F1@0.50': tensor(0.2221)}


In [13]:
# Experiment 6    {'batch': 16, 'lr': 0.001, 'optimizer': 'SGD'}
model = models.get("yolo_nas_m", checkpoint_path="/home/computervision_zl/Documents/yolonas/notebook_ckpts/YOLONAS_6/ckpt_best.pth", num_classes=13)

CHECKPOINT_DIR = 'notebook_ckpts_test/'
trainer = Trainer(experiment_name=f'YOLONAS_6', ckpt_root_dir=CHECKPOINT_DIR)

print(
    trainer.test(
        model=model,
        test_loader=test_loader_16,
        test_metrics_list=DetectionMetrics_050(
            score_thres=0.1, 
            top_k_predictions=300, 
            num_cls=13, 
            normalize_targets=True, 
            post_prediction_callback=PPYoloEPostPredictionCallback(
                score_threshold=0.01, 
                nms_top_k=1000, 
                max_predictions=300,
                nms_threshold=0.7)
            )
        )
)
del trainer
sleep(5)

Test: 100%|██████████| 32/32 [00:04<00:00,  7.40it/s]


{'Precision@0.50': tensor(0.3356), 'Recall@0.50': tensor(0.9626), 'mAP@0.50': tensor(0.8992), 'F1@0.50': tensor(0.4737)}


In [14]:
# Experiment 7    {'batch': 16, 'lr': 0.01, 'optimizer': 'Adam'}
model = models.get("yolo_nas_m", checkpoint_path="/home/computervision_zl/Documents/yolonas/notebook_ckpts/YOLONAS_7/ckpt_best.pth", num_classes=13)

CHECKPOINT_DIR = 'notebook_ckpts_test/'
trainer = Trainer(experiment_name=f'YOLONAS_7', ckpt_root_dir=CHECKPOINT_DIR)

print(
    trainer.test(
        model=model,
        test_loader=test_loader_16,
        test_metrics_list=DetectionMetrics_050(
            score_thres=0.1, 
            top_k_predictions=300, 
            num_cls=13, 
            normalize_targets=True, 
            post_prediction_callback=PPYoloEPostPredictionCallback(
                score_threshold=0.01, 
                nms_top_k=1000, 
                max_predictions=300,
                nms_threshold=0.7)
            )
        )
)

del trainer
sleep(5)

Test: 100%|██████████| 32/32 [00:04<00:00,  7.00it/s]


{'Precision@0.50': tensor(0.0566), 'Recall@0.50': tensor(0.8112), 'mAP@0.50': tensor(0.3721), 'F1@0.50': tensor(0.0950)}


In [15]:
# Experiment 8    {'batch': 16, 'lr': 0.01, 'optimizer': 'SGD'}
model = models.get("yolo_nas_m", checkpoint_path="/home/computervision_zl/Documents/yolonas/notebook_ckpts/YOLONAS_8/ckpt_best.pth", num_classes=13)

CHECKPOINT_DIR = 'notebook_ckpts_test/'
trainer = Trainer(experiment_name=f'YOLONAS_8', ckpt_root_dir=CHECKPOINT_DIR)

print(
    trainer.test(
        model=model,
        test_loader=test_loader_16,
        test_metrics_list=DetectionMetrics_050(
            score_thres=0.1, 
            top_k_predictions=300, 
            num_cls=13, 
            normalize_targets=True, 
            post_prediction_callback=PPYoloEPostPredictionCallback(
                score_threshold=0.01, 
                nms_top_k=1000, 
                max_predictions=300,
                nms_threshold=0.7)
            )
        )
)

del trainer
sleep(5)

Test: 100%|██████████| 32/32 [00:04<00:00,  7.35it/s]


{'Precision@0.50': tensor(0.2715), 'Recall@0.50': tensor(0.9604), 'mAP@0.50': tensor(0.8933), 'F1@0.50': tensor(0.4033)}
